In [1]:
import pandas as pd
import geopandas as gpd
from pandas_geojson import to_geojson
from shapely.geometry import Point
import json


In [2]:
#Create path and red in CSV
path = 'Wildfires_2.csv'
csv = pd.read_csv(path)
#Convert CSV into DF
df = pd.DataFrame(csv)
df.head()

,index,FIRE_YEAR,Fire_Date,STAT_CAUSE_DESCR,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE
0,1,2004,2004-05-12,Lightning,1530.0,0.25,A,38.933056,-120.404444,CA
1,3,2004,2004-06-28,Lightning,1400.0,0.10,A,38.559167,-119.913333,CA
2,4,2004,2004-06-28,Lightning,1200.0,0.10,A,38.559167,-119.933056,CA
3,5,2004,2004-06-30,Lightning,1600.0,0.10,A,38.635278,-120.103611,CA
4,6,2004,2004-07-01,Lightning,1400.0,0.10,A,38.688333,-120.153333,CA


In [3]:
#Create a GeoDataFrame with geometry column
geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]
geo = gpd.GeoDataFrame(df, geometry=geometry)
geo

,index,FIRE_YEAR,Fire_Date,STAT_CAUSE_DESCR,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,geometry
0,1,2004,2004-05-12,Lightning,1530.0,0.25,A,38.933056,-120.404444,CA,POINT (-120.40444 38.93306)
1,3,2004,2004-06-28,Lightning,1400.0,0.10,A,38.559167,-119.913333,CA,POINT (-119.91333 38.55917)
2,4,2004,2004-06-28,Lightning,1200.0,0.10,A,38.559167,-119.933056,CA,POINT (-119.93306 38.55917)
3,5,2004,2004-06-30,Lightning,1600.0,0.10,A,38.635278,-120.103611,CA,POINT (-120.10361 38.63528)
4,6,2004,2004-07-01,Lightning,1400.0,0.10,A,38.688333,-120.153333,CA,POINT (-120.15333 38.68833)
...,...,...,...,...,...,...,...,...,...,...,...
278463,1880370,2010,2010-07-26,Lightning,1351.0,81.00,C,41.508867,-120.819167,CA,POINT (-120.81917 41.50887)
278464,1880391,2010,2010-07-27,Lightning,NaN,1.00,B,41.684917,-122.716183,CA,POINT (-122.71618 41.68492)
278465,1880434,2015,2015-08-06,Lightning,1935.0,0.50,B,39.655817,-121.234567,CA,POINT (-121.23457 39.65582)
278466,1880448,2015,2015-06-26,Lightning,2014.0,0.05,A,40.443000,-120.664433,CA,POINT (-120.66443 40.44300)


In [4]:
#Drop unnecesary columns
gdf = geo.drop(['LATITUDE', 'LONGITUDE', 'index'], axis = 1)

In [5]:
#Convert non-spatial columns to properties
properties = gdf.drop(['geometry'], axis = 1).to_dict(orient='records')

# Convert Point geometries to GeoJSON-compatible format
gdf['geometry'] = gdf['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})

#Add properties to GeoJSON
geojson_data = {
    "features": []
}

for feature, prop in zip(gdf['geometry'], properties):
    geojson_data['features'].append({
        "geometry": feature,
        "properties": prop
    })
    
# Save GeoJSON to a file
with open('output.geojson', 'w') as f:
    json.dump(geojson_data, f, indent=2)

d:\Anaconda\envs\dev\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
